In [ ]:
!pip install SentenceTransformer

In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses
import pandas as pd
import numpy as np

# Load the SBERT model
model = SentenceTransformer('bert-base-nli-mean-tokens')

# Load the training data into a pandas dataframe
df = pd.read_csv('training_data.csv')
test_df = pd.read_csv('test.csv')

# Create a list of InputExample objects
examples = []
for i, row in df.iterrows():
    examples.append(InputExample(texts=[row['text_a'], row['text_b']], label=row['score']))


test_examples = []
for i, row in test_df.iterrows():
    test_examples.append(InputExample(texts=[row['sentence1'], row['sentence2']], label=row['label']))

# Define the training data generator
def data_generator(examples, batch_size=16):
    for i in range(0, len(examples), batch_size):
        batch = examples[i:i+batch_size]
        texts = [example.texts for example in batch]
        labels = [example.label for example in batch]
        yield texts, labels

# Define the training loss function
train_loss = losses.CosineSimilarityLoss(model)

# Fine-tune the model
model.fit(
    data_generator(examples),
    epochs=5,
    steps_per_epoch=int(np.ceil(len(examples) / 16)),
    warmup_steps=100,
    loss=train_loss,
)

# Save the fine-tuned model
model.save('finetuned_model')


In [ ]:
from sklearn.metrics import classification_report

# Load the fine-tuned model
model = SentenceTransformer('finetuned_model')

# Encode the test examples
test_embeddings = model.encode([(example.texts[0], example.texts[1]) for example in test_examples])

# Compute the cosine similarity scores between the test embeddings
similarity_scores = []
for i in range(len(test_embeddings)):
    score = cosine_similarity([test_embeddings[i][0]], [test_embeddings[i][1]])[0][0]
    similarity_scores.append(score)

# Compute the classification report
y_true = [example.label for example in test_examples]
y_pred = [1 if score >= 0.5 else 0 for score in similarity_scores]
print(classification_report(y_true, y_pred))